# Generating the model response

You have to run the docker image and bind its port to 8080

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
response = requests.get('http://localhost:8080/api/intent?sentence=Ceci%est%un%test')

# Printing json response
print(response.json())

{'find-train': 0.05815774202346802, 'irrelevant': 0.5158529281616211, 'find-flight': 0.057573966681957245, 'find-restaurant': 0.07580501586198807, 'purchase': 0.08574576675891876, 'find-around-me': 0.07513963431119919, 'provide-showtimes': 0.06499148905277252, 'find-hotel': 0.06673342734575272}


In [3]:
# The sum of all the predictions is equal to ~1. This means that the model outputs a softmax result
print(sum(response.json().values()))

0.9999999701976776


In [4]:
# Read training data or testing data
df = pd.read_json('testing_set.json')

In [7]:
data = []
class_names = list(response.json().keys())

for index in tqdm(df.index, total=len(df)):
    sentence = df.iloc[index].sentence
    intent = df.iloc[index].intent
    res = requests.get(f'http://localhost:8080/api/intent?sentence={sentence}')
    i = np.argmax(list(res.json().values()))
    max_prediction = class_names[i]
    data.append({'intent': intent, 'max_prediction': max_prediction, **res.json()})

    # if index > 20:
    #     break

100%|██████████| 1065/1065 [00:15<00:00, 69.74it/s]


In [6]:
pd_data = pd.DataFrame(data)
print(pd_data.head())

pd_data.to_json('testing_model_predictions.json')

            intent   max_prediction  find-train  irrelevant  find-flight  \
0       irrelevant         purchase    0.019063    0.388085     0.027179   
1       irrelevant       irrelevant    0.033228    0.542000     0.027649   
2       irrelevant       irrelevant    0.017024    0.704971     0.020258   
3         purchase       irrelevant    0.019862    0.422897     0.023757   
4  find-restaurant  find-restaurant    0.029981    0.184255     0.009827   

   find-restaurant  purchase  find-around-me  provide-showtimes  find-hotel  
0         0.053374  0.398776        0.074593           0.010972    0.027957  
1         0.029301  0.089124        0.062782           0.034890    0.181026  
2         0.102957  0.057472        0.034300           0.025665    0.037352  
3         0.027854  0.286700        0.131648           0.016949    0.070332  
4         0.581406  0.127771        0.031913           0.021814    0.013032  
